In [1]:
# use an nvidia shell command to check the gpu
! nvidia-smi

Tue Apr 18 09:27:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# N8 Python GPU Workshop April 2023



## Our first GPU Kernel

In [45]:
# vector_add that adds two input vectors together and stores the sum in another vector

def vector_add(A, B, C, size):
  for item in range(0, size):
    C[item] = A[item] + B[item]

In [46]:
import numpy as np

size = 1024

a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

vector_add(a_cpu, b_cpu, c_cpu, size)

In [47]:
print(c_cpu)

[1.31055669 0.64728473 1.26046307 ... 0.89564628 1.30866273 1.8148962 ]


In [48]:
# converting the above code into a CUDA kernel

import cupy 

# CUDA kernel

vector_add_cuda_code = r'''
extern "C"
__global__ void vector_add(const float * A, const float * B, float * C, const int size)
{
  int item = threadIdx.x;
  C[item] = A[item] + B[item];
}
'''

In [49]:
# compile the kernel into a function
# we can run, through CuPy

vector_add_gpu = cupy.RawKernel(vector_add_cuda_code, "vector_add")

In [50]:
# run the GPU version of vector_add

size = 1024

# create cupy arrays from numpy arrays
a_gpu = cupy.asarray(a_cpu, dtype=cupy.float32)
b_gpu = cupy.asarray(b_cpu, dtype=cupy.float32)
c_gpu = cupy.zeros(size, dtype=cupy.float32)

In [51]:
# run our custom kernel on the GPU

# specified 3 tuples, grid configuration, block configuration, kernel arguments
vector_add_gpu((1, 1, 1), (size, 1, 1), (a_gpu, b_gpu, c_gpu, size))

In [52]:
if np.allclose(c_cpu, c_gpu):
  print("Correct results!")

Correct results!


In [53]:
# using data larger than 1024

size = 2048

a_gpu = cupy.random.rand(size, dtype=cupy.float32)
b_gpu = cupy.random.rand(size, dtype=cupy.float32)
c_gpu = cupy.zeros(size, dtype=cupy.float32)

In [54]:
# this errors because we've requested more than 1024 threads per block
# in the second tuple argument
vector_add_gpu((1,1,1), (size, 1, 1), (a_gpu, b_gpu, c_gpu, size))

CUDADriverError: ignored

In [55]:
# can we get around the block size limit
# by request 2 blocks with half the size as number of threads

vector_add_gpu((2,1,1), (size // 2, 1, 1), (a_gpu, b_gpu, c_gpu, size))

In [56]:
# test if this has worked as expected

a_cpu = cupy.asnumpy(a_gpu)
b_cpu = cupy.asnumpy(b_gpu)
c_cpu = np.zeros(size, dtype=np.float32)

# call the python version of vector add
vector_add(a_cpu, b_cpu, c_cpu, size)

In [57]:
if np.allclose(c_cpu, c_gpu):
  print("Right results!")
else:
  print("Wrong results!")

Wrong results!


In [58]:
print(c_gpu)

[1.0210285 1.1107659 1.0539565 ... 0.        0.        0.       ]


In [59]:
print(c_cpu)

[1.0210285  1.1107659  1.0539565  ... 0.9826169  0.89512354 1.0370884 ]


In [60]:
# CUDA kernel that accounts for multiple blocks

# we need to use additional CUDA variables
# to help calculate the right index for each thread in each block


vector_add_cuda_code = r'''
extern "C"
__global__ void vector_add(const float * A, const float * B, float * C, const int size)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;
  C[item] = A[item] + B[item];
}
'''

In [61]:
vector_add_gpu = cupy.RawKernel(vector_add_cuda_code, "vector_add")

vector_add_gpu((2, 1, 1), (size // 2, 1, 1), (a_gpu, b_gpu, c_gpu, size))

In [62]:
if np.allclose(c_cpu, c_gpu):
  print("All correct!")
else:
  print("Wrong results!")

All correct!


In [63]:
# updating the kernel to handle arbitrary values

vector_add_cuda_code = r'''
extern "C"
__global__ void vector_add(const float * A, const float * B, float * C, const int size)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;
  if ( item < size )
  {
    C[item] = A[item] + B[item];
  }
}
'''

In [64]:
# using data of an arbitrary size

size = 10_000

a_gpu = cupy.random.rand(size, dtype=cupy.float32)
b_gpu = cupy.random.rand(size, dtype=cupy.float32)
c_gpu = cupy.zeros(size, dtype=cupy.float32)

In [65]:
vector_add_gpu = cupy.RawKernel(vector_add_cuda_code, "vector_add")

vector_add_gpu((2, 1, 1), (size // 2, 1, 1), (a_gpu, b_gpu, c_gpu, size))

CUDADriverError: ignored

In [66]:
# calculating the required number of blocks
# in our CUDA grid to run data of arbitrary size

import math 

# specify that our blocks should always have
# 1024 threads in the x dimension
block_size = (1024, 1, 1)

grid_size = (int(math.ceil(size / 1024)), 1, 1)

In [67]:
print(grid_size)

(10, 1, 1)


In [68]:
# now we can run our vector_add code on data of an arbitrary size
# using Python logic to help calculate the CUDA grid
vector_add_gpu(grid_size, block_size, (a_gpu, b_gpu, c_gpu, size))

In [69]:
a_cpu = cupy.asnumpy(a_gpu)
b_cpu = cupy.asnumpy(b_gpu)
c_cpu = np.zeros(size, dtype=np.float32)

vector_add(a_cpu, b_cpu, c_cpu, size)

In [70]:
# check if our results match

if np.allclose(c_cpu, c_gpu):
  print("Correct results!")
else:
  print("Oh no!")

Correct results!


## Registers, local and global memory

In [ ]:
# making register use more explicit
# using temp variables to show register use

vector_add_cuda_code = r'''
extern "C"
__global__ void vector_add(const float * A, const float * B, float * C, const int size)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;
  float temp_a, temp_b, temp_c;

  if ( item < size )
  {
    temp_a = A[item];
    temp_b = B[item];
    temp_c = temp_a + temp_b;
    C[item] = temp_c;
  }
}
'''

In [ ]:
# making register use more explicit
# using temp array to show register use

vector_add_cuda_code = r'''
extern "C"
__global__ void vector_add(const float * A, const float * B, float * C, const int size)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;
  float temp[3];

  if ( item < size )
  {
    temp[0] = A[item];
    temp[1] = B[item];
    temp[2] = temp[0] + temp[1];
    C[item] = temp[2];
  }
}
'''

In [ ]:
# using local memory on the thread

vector_add_cuda_code = r'''
extern "C"
__global__ void vector_add(const float * A, const float * B, float * C, const int size, const int local_mem_size)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;
  float local_memory[local_mem_size];

  if ( item < size )
  {
    local_memory[0] = A[item];
    local_memory[1] = B[item];
    local_memory[2] = local_memory[0] + local_memory[1];
    C[item] = local_memory[2];
  }
}
'''

# vector_add_gpu((2,1,1), (size // 2, 1, 1), (a_gpu, b_gpu, c_gpu, size, 3)

In [44]:
# checking shared memory with larger than block data

# using data of an arbitrary size

size = 2048

a_gpu = cupy.random.rand(size, dtype=cupy.float32)
b_gpu = cupy.random.rand(size, dtype=cupy.float32)
c_gpu = cupy.zeros(size, dtype=cupy.float32)

# using shared memory

vector_add_cuda_code_shared = r'''
extern "C"
__global__ void vector_add(const float * A, const float * B, float * C, const int size)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;
  __shared__ float temp[3];

  if ( item < size )
  {
    temp[0] = A[item];
    temp[1] = B[item];
    temp[2] = temp[0] + temp[1];
    C[item] = temp[2];
  }
}
'''
threads_per_block = 32
grid_size = ( int(math.ceil(size / threads_per_block)), 1, 1)
block_size = (threads_per_block, 1, 1)
vector_add_gpu_shared = cupy.RawKernel(vector_add_cuda_code_shared, "vector_add")

# where we're running our CUDA kernel
vector_add_gpu_shared(
    # specifying grid size
    grid_size,
    # specify block size
    block_size,
    # specify gpu function arguments 
    (a_gpu, b_gpu, c_gpu, size)
)

a_cpu = cupy.asnumpy(a_gpu)
b_cpu = cupy.asnumpy(b_gpu)
c_cpu = np.zeros(size, dtype=np.float32)

vector_add(a_cpu, b_cpu, c_cpu, size)

if np.allclose(c_gpu, c_gpu):
  print("All good")
else:
  print("Oh no")

All good


In [31]:
# using shared memory

vector_add_cuda_code_shared = r'''
extern "C"
__global__ void vector_add(const float * A, const float * B, float * C, const int size)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;
  int offset = threadIdx.x * 3;
  extern __shared__ float temp[];

  if ( item < size )
  {
    temp[offset + 0] = A[item];
    temp[offset + 1] = B[item];
    temp[offset + 2] = temp[offset + 0] + temp[offset + 1];
    C[item] = temp[offset + 2];
  }
}
'''

In [32]:
import cupy

vector_add_gpu_shared = cupy.RawKernel(vector_add_cuda_code_shared, "vector_add")

In [33]:
# using data of an arbitrary size

size = 2048

a_gpu = cupy.random.rand(size, dtype=cupy.float32)
b_gpu = cupy.random.rand(size, dtype=cupy.float32)
c_gpu = cupy.zeros(size, dtype=cupy.float32)

In [34]:
# add section for handling how many blocks to request

import math

threads_per_block = 32
grid_size = ( int(math.ceil(size / threads_per_block)), 1, 1)
block_size = (threads_per_block, 1, 1)

In [35]:
print(grid_size)

(64, 1, 1)


In [36]:
# where we're running our CUDA kernel
vector_add_gpu_shared(
    # specifying grid size
    grid_size,
    # specify block size
    block_size,
    # specify gpu function arguments 
    (a_gpu, b_gpu, c_gpu, size), 
    # specify how much shared memory is required per block
    shared_mem=(threads_per_block * 3 * cupy.dtype(cupy.float32).itemsize)
)

In [37]:
print(c_gpu)

[1.0004283  1.1307968  0.91168463 ... 0.6923103  0.97360146 1.6778005 ]


In [38]:
import numpy as np

a_cpu = cupy.asnumpy(a_gpu)
b_cpu = cupy.asnumpy(b_gpu)
c_cpu = np.zeros(size, dtype=np.float32)

vector_add(a_cpu, b_cpu, c_cpu, size)

In [39]:
if np.allclose(c_gpu, c_gpu):
  print("All good")
else:
  print("Oh no")

All good


In [74]:
# lets benchmark vector_add_gpu v vector_add_gpu_shared

from cupyx.profiler import benchmark 

size = 2048

a_gpu = cupy.random.rand(size, dtype=cupy.float32)
b_gpu = cupy.random.rand(size, dtype=cupy.float32)
c_gpu = cupy.zeros(size, dtype=cupy.float32)

threads_per_block = 32
grid_size = ( int(math.ceil(size / threads_per_block)), 1, 1)
block_size = (threads_per_block, 1, 1)

execution_gpu = benchmark(vector_add_gpu,
                          (grid_size, block_size, (a_gpu, b_gpu, c_gpu, size)), 
                          n_repeat=10)

c_gpu = cupy.zeros(size, dtype=cupy.float32)

execution_gpu_shared = benchmark(vector_add_gpu_shared,
                                 (grid_size, block_size, (a_gpu, b_gpu, c_gpu, size)),
                                 n_repeat=10)
 

print(f"vector_add_gpu took an average of: {np.average(execution_gpu.gpu_times):.6f}")

print(f"vector_add_gpu_shared took an average of: {np.average(execution_gpu_shared.gpu_times):.6f}")

vector_add_gpu took an average of: 0.000024
vector_add_gpu_shared took an average of: 0.000025


## Shared memory and a histogram function

In [75]:
def histogram(input_array, output_array):
  for item in input_array:
    output_array[item] = output_array[item] + 1   

In [76]:
input_array = np.random.randint(256, size=2048, dtype=np.int32)

output_array = np.zeros(256, dtype=np.int32)

histogram(input_array, output_array)

In [77]:
print(output_array)

[ 7  5  6 10  6 10 10 12  6  7  7 10  6  5 11  9  4 11  8  4 11  5  5 16
  8 10  7 10  7 11  8 10  5  9  5  5  8  9  9  9 10 10  9  5 13  9  4  9
 10  8 13  7  6  6  5  6  8 12  8 11  5  5  7  6  7  8  7 10  9  3 16  5
 12  9  6  8 11 11 11  5  1  7  7  5 12  8  5 12  6  9 13  6  9  7 14  7
  4  7  7  6 10  7 11  6  5 14  9  9  8 10  7 11  6 13  7  9  8  4  7  7
  9  8 15  7  5 12 10  8 10  6  7 10  6 10  6  8  5 10  8  2  4  3  9 11
  9 10 12 13 11 13  7 12  3 10  4  9  4 10  4  8 10  9  9 10 13  8  5 19
  8 14  9  7  8  6  9  5  8 15 10  7  7  8  4  6  6 11  7  8  8  7  8 10
 11  5  5  5 10  9  5  2  7  7  5 13  2  6  2 10  4  6 12  9 10  6  8  8
  6  2  8 15 11  9  5  3  3  9  4  6  7 11  5  6  5 12  8 10 15  5 12 10
  6  7  6  6  8  3 10  7  9  7  9  6  9  8  9  6]


In [ ]:
histogram_cuda = r'''
extern "C"
__global__ void histogram(const int * input, int * output)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;

  output[input[item]] = output[input[item]] + 1

'''

In [ ]:
# [7, 5, 4, 7]
# thread 0 - 7
# thread 1 - 5
# thread 2 - 4
# thread 3 - 7

# this will create a race condition where threads 0 and 3 try to update output[7]
# at the same time overwriting one another

In [83]:
histogram_cuda = r'''
extern "C"
__global__ void histogram(const int * input, int * output)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;

  atomicAdd(&(output[input[item]]), 1);
}
'''

In [85]:
# test the GPU code versus the python code 

size = 2 ** 25

# create some input data for cpu and gpu

input_gpu = cupy.random.randint(256, size=size, dtype=cupy.int32)
input_cpu = cupy.asnumpy(input_gpu)
output_gpu = cupy.zeros(256, dtype=cupy.int32)
output_cpu = cupy.asnumpy(output_gpu)

# compile and setup CUDA kernel

histogram_gpu = cupy.RawKernel(histogram_cuda, "histogram")
threads_per_block = 256 
grid_size = (int(math.ceil(size / threads_per_block)), 1, 1)
block_size = (threads_per_block, 1, 1)

# correctness check

histogram(input_cpu, output_cpu)
histogram_gpu(grid_size, block_size, (input_gpu, output_gpu))
if np.allclose(output_cpu, output_gpu):
  print("Correct results!")
else:
  print("Oh no!")

# measure performance

%timeit -n 1 -r 1 histogram(input_cpu, output_cpu)
execution_gpu = benchmark(histogram_gpu, 
                          (grid_size, block_size, (input_gpu, output_gpu)), 
                           n_repeat=10)
gpu_avg_time = np.average(execution_gpu.gpu_times)
print(f"{gpu_avg_time:.6f} s")

Correct results!
1min 25s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
0.013137 s


In [90]:
# using shared memory to prevent conflicts in global memory

histogram_cuda_shared = r'''
extern "C"
__global__ void histogram(const int * input, int * output)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;
  __shared__ int temp_histogram[256];

  // initialise shared memory
  temp_histogram[threadIdx.x] = 0;
  __syncthreads();

  // update block shared memory for value in item
  atomicAdd(&(temp_histogram[input[item]]), 1);
  __syncthreads();

  // update the global histogram (output) using temporary histogram
  atomicAdd(&(output[threadIdx.x]), temp_histogram[threadIdx.x]);
}
'''

In [91]:
# test the GPU code using shared memory versus the python code 

size = 2 ** 25

# create some input data for cpu and gpu

input_gpu = cupy.random.randint(256, size=size, dtype=cupy.int32)
input_cpu = cupy.asnumpy(input_gpu)
output_gpu = cupy.zeros(256, dtype=cupy.int32)
output_cpu = cupy.asnumpy(output_gpu)

# compile and setup CUDA kernel

histogram_gpu_shared = cupy.RawKernel(histogram_cuda_shared, "histogram")
threads_per_block = 256 
grid_size = (int(math.ceil(size / threads_per_block)), 1, 1)
block_size = (threads_per_block, 1, 1)

# correctness check

histogram(input_cpu, output_cpu)
histogram_gpu(grid_size, block_size, (input_gpu, output_gpu))
histogram_gpu_shared(grid_size, block_size, (input_gpu, output_gpu))
# worth noting that output_gpu is generated from _shared not _gpu 
if np.allclose(output_cpu, output_gpu):
  print("Correct results!")
else:
  print("Oh no!")

# measure performance

%timeit -n 1 -r 1 histogram(input_cpu, output_cpu)
execution_gpu = benchmark(histogram_gpu, 
                          (grid_size, block_size, (input_gpu, output_gpu)), 
                           n_repeat=10)
execution_gpu_shared = benchmark(histogram_gpu_shared, 
                          (grid_size, block_size, (input_gpu, output_gpu)), 
                           n_repeat=10)
gpu_avg_time = np.average(execution_gpu.gpu_times)
gpu_avg_time_shared = np.average(execution_gpu_shared.gpu_times)
print(f"{gpu_avg_time:.6f} s")
print(f"{gpu_avg_time_shared:.6f} s")

Oh no!
1min 39s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
0.014929 s
0.001898 s


## Constant data on GPU

In [95]:
# we can define constant data that is accessible to all threads but is read-only

constant_kernel = r'''
extern "C" {
#define BLOCKS 2 

__constant__ float factors[BLOCKS];

__global__ void sum_and_multiply(const float * A, const float * B, float * C, const int size)
{
  int item = (blockIdx.x * blockDim.x) + threadIdx.x;
  C[item] = (A[item] + B[item]) * factors[blockIdx.x];
}
}
'''

In [96]:
module = cupy.RawModule(code = constant_kernel)

In [97]:
# get a pointer to the memory location of the constant
# variable factors
factor_ptr = module.get_global("factors")

In [98]:
# create some data on the host
factors_gpu = cupy.ndarray(2, cupy.float32, factor_ptr)

In [99]:
factors_gpu[...] = cupy.random.random(2, dtype=cupy.float32)

## Concurrency on GPU

In [100]:
vector_add_gpu

In [101]:
stream_one = cupy.cuda.Stream() 
stream_two = cupy.cuda.Stream()

In [102]:
# use streams to run two kernels without one depending on the other

c_gpu = cupy.zeros(size, dtype=cupy.float32)

with stream_one:
  histogram_gpu_shared(grid_size, block_size, (input_gpu, output_gpu))

with stream_two: 
  histogram_gpu(grid_size, block_size, (input_gpu, output_gpu))

In [ ]:
# using cuda events to manage concurrent kernel runs

stream_one = cupy.cuda.Stream()
stream_two = cupy.cuda.Stream()
sync_point = cupy.cuda.Event()

with stream_one:
  histogram_gpu(grid_size, block_size, (input_gpu, output_gpu))
  sync_point.record(stream=stream_one)
  histogram_gpu(grid_size, block_size, (input_gpu, output_gpu))

with stream_two:
  stream_two.wait_event(sync_point)
  histogram_gpu(grid_size, block_size, (input_gpu, output_gpu))
